In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [3]:

main_webUrl = "https://www.rumah123.com/jual/dki-jakarta/rumah/?page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [4]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [5]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

2500

In [6]:
dataframe = {}
dataframe["Nama"] = []
dataframe["Harga"] = []

In [ ]:
for i in range(1,200):
    webUrl = f"https://www.rumah123.com/jual/dki-jakarta/rumah/?page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue
        
        print(link)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe.keys():
                dataframe[prop_key] = []
            
            if prop_key in dataframe.keys():
                dataframe[prop_key].append(prop_value)
            
        print("")


/properti/jakarta-utara/hos16937294/

/properti/jakarta-utara/hos19295303/

/properti/jakarta-utara/hos15621647/

/properti/jakarta-pusat/hos16893264/

/properti/jakarta-selatan/hos19174759/

/properti/jakarta-selatan/hos15702488/

/properti/jakarta-barat/hos17603112/

/properti/jakarta-selatan/hos19174774/

/properti/jakarta-barat/hos19389867/

/properti/jakarta-barat/hos19179156/

/properti/jakarta-utara/hos19034929/

/properti/jakarta-barat/hos18306821/

/properti/jakarta-barat/hos19295213/

/properti/jakarta-utara/hos15833746/

/properti/jakarta-barat/hos19182513/

/properti/jakarta-utara/hos19142154/

/properti/jakarta-selatan/hos19054371/

/properti/jakarta-selatan/hos19050195/

/properti/jakarta-selatan/hos17233142/

/properti/jakarta-selatan/hos18857823/

/properti/jakarta-utara/hos11774682/

/properti/jakarta-timur/hos19030862/

/properti/jakarta-timur/hos17255638/

/properti/jakarta-selatan/hos15457706/

/properti/jakarta-barat/hos18376875/

/properti/jakarta-selatan/hos19014

In [ ]:
rumah123_df = pd.DataFrame.from_dict(dataframe, orient='index').T
rumah123_df = rumah123_df.fillna(0)
rumah123_df.to_json("Provinsi_Jakarta.json", orient="records")